<a href="https://colab.research.google.com/github/scardenol/RP_argos/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import sympy as sp

In [ ]:
def Taylor_polynomial_sympy(function_expression, variable_list, evaluation_point, degree):
    """
    Mathematical formulation reference:
    https://math.libretexts.org/Bookshelves/Calculus/Supplemental_Modules_(Calculus)/Multivariable_Calculus/3%3A_Topics_in_Partial_Derivatives/Taylor__Polynomials_of_Functions_of_Two_Variables
    :param function_expression: Sympy expression of the function
    :param variable_list: list. All variables to be approximated (to be "Taylorized")
    :param evaluation_point: list. Coordinates, where the function will be expressed
    :param degree: int. Total degree of the Taylor polynomial
    :return: Returns a Sympy expression of the Taylor series up to a given degree, of a given multivariate expression, approximated as a multivariate polynomial evaluated at the evaluation_point
    """
    from sympy import factorial, Matrix, prod
    import itertools

    n_var = len(variable_list)
    point_coordinates = [(i, j) for i, j in (zip(variable_list, evaluation_point))]  # list of tuples with variables and their evaluation_point coordinates, to later perform substitution

    deriv_orders = list(itertools.product(range(degree + 1), repeat=n_var))  # list with exponentials of the partial derivatives
    deriv_orders = [deriv_orders[i] for i in range(len(deriv_orders)) if sum(deriv_orders[i]) <= degree]  # Discarding some higher-order terms
    n_terms = len(deriv_orders)
    deriv_orders_as_input = [list(sum(list(zip(variable_list, deriv_orders[i])), ())) for i in range(n_terms)]  # Individual degree of each partial derivative, of each term

    polynomial = 0
    for i in range(n_terms):
        partial_derivatives_at_point = function_expression.diff(*deriv_orders_as_input[i]).subs(point_coordinates)  # e.g. df/(dx*dy**2)
        denominator = prod([factorial(j) for j in deriv_orders[i]])  # e.g. (1! * 2!)
        distances_powered = prod([(Matrix(variable_list) - Matrix(evaluation_point))[j] ** deriv_orders[i][j] for j in range(n_var)])  # e.g. (x-x0)*(y-y0)**2
        polynomial += partial_derivatives_at_point / denominator * distances_powered
    return polynomial

In [ ]:
def Model_taylor_sympy(taylor_serie):
  """
  param taylor_serie: taylor serie to be converted to model
  return: list of model termns
  """
  from sympy import symbols, init_printing, Function, sympify, Derivative, simplify, ordered

  # Convert Serie to list
  S = list(sympify((sympify(taylor_serie, evaluate=False)).args))
  M = S.copy() # Copy the list to use for the model

  # Check for derivatives
  # Substitue derivatives from Serie List with parameters (cleaner than from Serie)
  d = [list(i.atoms(Derivative)) for i in M]
  d_unpacked = [x for l in d for x in l] # this unpacks it but gets messy if a list had more than 1 element
  
  # if d_unpacked is empty it means there are no derivatives in the serie
  B = [];

  if len(d_unpacked) != 0:  # if there are derivatives
    indexes = [idx for idx in range(len(d)) if len(d[idx])] # Get indexes

    for i in indexes:
      B_aux = sympify(['b' + str(b) for b in range(len(B), len(B) + 1)])
      B += B_aux
      M[i] = simplify(M[i].subs(list(ordered(d[i]))[0], B_aux[0]))
    
    # Clean the f(c) termn at the end
    B_aux = sympify(['b' + str(b) for b in range(len(B), len(B) + 1)])
    B += B_aux
    M[-1] = B_aux[0]

    # Multiply resulting list with the parameter list A (excluding the last element which has only 1 parameter)
    A = sympify(['a' + str(a) for a in range(0, len(B)-1)])
    M = [a*m for a,m in zip(A,M[:-1])] + [M[-1]]
  
  else: # If there are not derivatives
    # Multiply resulting list with the parameter list A
    A = sympify(['a' + str(a) for a in range(0, len(M))])
    M = [a*m for a,m in zip(A,M)]
  
  return M, A, B

In [ ]:
from sympy import symbols, init_printing, Function, sympify, Derivative, simplify, ordered

init_printing(use_latex='mathjax') # Printing preferences

# Model F(x1,x2)
x1, x2, x3 = symbols('x1, x2, x3')
f = Function('f')
function_expression = f(x1*x2*x3)
variable_list = [x1,x2,x3]
evaluation_point = [1, 2, 3]
degree=2
F = Taylor_polynomial_sympy(function_expression, variable_list, evaluation_point, degree)
display(F)


                                                                              
                                                                              
                                                                              
             ⎛  2        ⎞│                         ⎛                         
           2 ⎜ d         ⎟│                         ⎜  ⎛ d        ⎞│          
18⋅(x₁ - 1) ⋅⎜────(f(ξ₁))⎟│     + (x₁ - 1)⋅(x₂ - 2)⋅⎜3⋅⎜───(f(ξ₁))⎟│     + 18⋅
             ⎜   2       ⎟│                         ⎜  ⎝dξ₁       ⎠│ξ₁=6      
             ⎝dξ₁        ⎠│ξ₁=6                     ⎝                         

                                                                              
                                                                              
                                                                              
⎛  2        ⎞│    ⎞                     ⎛                         ⎛  2        
⎜ d         ⎟│    ⎟                     ⎜  ⎛ d     

In [ ]:
# Generate the model
M, A, B = Model_taylor_sympy(F)

In [ ]:
# Display results
F_list = list(sympify((sympify(F, evaluate=False)).args))
display(F_list) # Original
display(M) # model
print(len(F_list) == len(M))
display(A, B) # Parameters
parameters = A + B
display(parameters) # List of all parameters

⎡                                                                             
⎢                                                                             
⎢                                                                             
⎢            ⎛  2        ⎞│                                                   
⎢          2 ⎜ d         ⎟│                 ⎛ d        ⎞│                 ⎛ d 
⎢2⋅(x₃ - 3) ⋅⎜────(f(ξ₁))⎟│    , 2⋅(x₃ - 3)⋅⎜───(f(ξ₁))⎟│    , 3⋅(x₂ - 2)⋅⎜───
⎢            ⎜   2       ⎟│                 ⎝dξ₁       ⎠│ξ₁=6             ⎝dξ₁
⎣            ⎝dξ₁        ⎠│ξ₁=6                                               

                                                                              
                                                                              
                                                                              
                                                          ⎛  2        ⎞│      
       ⎞│                 ⎛ d        ⎞│            

⎡                                                                             
⎢                2                                                            
⎢2⋅a₀⋅b₀⋅(x₃ - 3) , 2⋅a₁⋅b₁⋅(x₃ - 3), 3⋅a₂⋅b₂⋅(x₂ - 2), 6⋅a₃⋅b₃⋅(x₁ - 1), 18⋅a
⎣                                                                             

                                2                                             
             2  9⋅a₅⋅b₅⋅(x₂ - 2)                                              
₄⋅b₄⋅(x₁ - 1) , ─────────────────, 2⋅a₆⋅b₆⋅(x₁ - 1)⋅(x₃ - 3), 3⋅a₇⋅b₇⋅(x₁ - 1)
                        2                                                     

                                      ⎤
                                      ⎥
⋅(x₂ - 2), a₈⋅b₈⋅(x₂ - 2)⋅(x₃ - 3), b₉⎥
                                      ⎦

True


[a₀, a₁, a₂, a₃, a₄, a₅, a₆, a₇, a₈]

[b₀, b₁, b₂, b₃, b₄, b₅, b₆, b₇, b₈, b₉]

[a₀, a₁, a₂, a₃, a₄, a₅, a₆, a₇, a₈, b₀, b₁, b₂, b₃, b₄, b₅, b₆, b₇, b₈, b₉]

In [ ]:
# Convert list back to sum
D = sum(M)
display(D)

                                                                              
                2                                                             
2⋅a₀⋅b₀⋅(x₃ - 3)  + 2⋅a₁⋅b₁⋅(x₃ - 3) + 3⋅a₂⋅b₂⋅(x₂ - 2) + 6⋅a₃⋅b₃⋅(x₁ - 1) + 1
                                                                              

                                    2                                         
                2   9⋅a₅⋅b₅⋅(x₂ - 2)                                          
8⋅a₄⋅b₄⋅(x₁ - 1)  + ───────────────── + 2⋅a₆⋅b₆⋅(x₁ - 1)⋅(x₃ - 3) + 3⋅a₇⋅b₇⋅(x
                            2                                                 

                                              
                                              
₁ - 1)⋅(x₂ - 2) + a₈⋅b₈⋅(x₂ - 2)⋅(x₃ - 3) + b₉
                                              

In [ ]:
vars = variable_list + parameters
display(vars) # Order of function parameters
fun = sp.lambdify(vars, D, 'scipy')
value = [i for i in range(len(vars))]
fun(*[1]*len(vars))

[x₁, x₂, x₃, a₀, a₁, a₂, a₃, a₄, a₅, a₆, a₇, a₈, b₀, b₁, b₂, b₃, b₄, b₅, b₆, b
₇, b₈, b₉]

8.5